<a href="https://colab.research.google.com/github/kalmuroth/lb-cml/blob/main/lb_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Random Forest for Fraud Detection with SciKit-Learn

In [12]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve, classification_report

import h2o
from h2o.frame import H2OFrame
from h2o.estimators.random_forest import H2ORandomForestEstimator

%matplotlib inline

ModuleNotFoundError: ignored

In [17]:
data = pd.read_csv('./drive/MyDrive/notebook/Fraud_Data.csv', parse_dates=['signup_time', 'purchase_time'])
data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [16]:
address2country = pd.read_csv('./drive/MyDrive/notebook/IpAddress_to_Country.csv')
address2country.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [27]:
# Merge the two datasets based on IP address range
merged_data = pd.concat([data, address2country], axis=1)
print(merged_data.head())

   user_id         signup_time       purchase_time  purchase_value  \
0    22058 2015-02-24 22:55:49 2015-04-18 02:47:11              34   
1   333320 2015-06-07 20:39:50 2015-06-08 01:38:54              16   
2     1359 2015-01-01 18:52:44 2015-01-01 18:52:45              15   
3   150084 2015-04-28 21:13:25 2015-05-04 13:54:50              44   
4   221365 2015-07-21 07:09:52 2015-09-09 18:40:53              39   

       device_id source browser sex  age    ip_address  class  \
0  QVPSPJUOCKZAR    SEO  Chrome   M   39  7.327584e+08      0   
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53  3.503114e+08      0   
2  YSSKYOSJHPPLJ    SEO   Opera   M   53  2.621474e+09      1   
3  ATGTXKYKUDUQN    SEO  Safari   M   41  3.840542e+09      0   
4  NAUITBZFJKHWW    Ads  Safari   M   45  4.155831e+08      0   

   lower_bound_ip_address  upper_bound_ip_address    country  
0              16777216.0              16777471.0  Australia  
1              16777472.0              16777727.0      China  

In [29]:
# Get the time difference between purchase time and signup time
merged_data['time_difference'] = merged_data['purchase_time'] - merged_data['signup_time']

# Create a new DataFrame with selected columns
new_dataframe = merged_data[['user_id', 'signup_time', 'purchase_time', 'purchase_value', 'device_id', 'source',
                             'browser', 'sex', 'age', 'ip_address', 'class', 'country', 'time_difference']]

# Print the first 5 rows of the new DataFrame
print(new_dataframe.head())


   user_id         signup_time       purchase_time  purchase_value  \
0    22058 2015-02-24 22:55:49 2015-04-18 02:47:11              34   
1   333320 2015-06-07 20:39:50 2015-06-08 01:38:54              16   
2     1359 2015-01-01 18:52:44 2015-01-01 18:52:45              15   
3   150084 2015-04-28 21:13:25 2015-05-04 13:54:50              44   
4   221365 2015-07-21 07:09:52 2015-09-09 18:40:53              39   

       device_id source browser sex  age    ip_address  class    country  \
0  QVPSPJUOCKZAR    SEO  Chrome   M   39  7.327584e+08      0  Australia   
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53  3.503114e+08      0      China   
2  YSSKYOSJHPPLJ    SEO   Opera   M   53  2.621474e+09      1      China   
3  ATGTXKYKUDUQN    SEO  Safari   M   41  3.840542e+09      0  Australia   
4  NAUITBZFJKHWW    Ads  Safari   M   45  4.155831e+08      0      China   

   time_difference  
0 52 days 03:51:22  
1  0 days 04:59:04  
2  0 days 00:00:01  
3  5 days 16:41:25  
4 50 days 11:31:0

In [31]:
# Check user number for unique devices
unique_devices_per_user = merged_data.groupby('user_id')['device_id'].nunique()
print(unique_devices_per_user)

user_id
2         1
4         1
8         1
9         1
12        1
         ..
399992    1
399993    1
399995    1
399997    1
400000    1
Name: device_id, Length: 151112, dtype: int64


In [32]:
# Check user number for unique ip_address
unique_ips_per_user = merged_data.groupby('user_id')['ip_address'].nunique()
print(unique_ips_per_user)

user_id
2         1
4         1
8         1
9         1
12        1
         ..
399992    1
399993    1
399995    1
399997    1
400000    1
Name: ip_address, Length: 151112, dtype: int64


In [34]:
# Signup day and week
# Purchase day and week
merged_data['signup_day'] = merged_data['signup_time'].dt.day_name()
merged_data['signup_week'] = merged_data['signup_time'].dt.isocalendar().week

# Extract purchase day and week
merged_data['purchase_day'] = merged_data['purchase_time'].dt.day_name()
merged_data['purchase_week'] = merged_data['purchase_time'].dt.isocalendar().week

# Print the first 5 rows with the new columns
print(merged_data[['user_id', 'signup_day', 'signup_week', 'purchase_day', 'purchase_week']].head())

   user_id signup_day  signup_week purchase_day  purchase_week
0    22058    Tuesday            9     Saturday             16
1   333320     Sunday           23       Monday             24
2     1359   Thursday            1     Thursday              1
3   150084    Tuesday           18       Monday             19
4   221365    Tuesday           30    Wednesday             37
